In [1]:
import requests, json
import pandas as pd
from bs4 import BeautifulSoup
import re
from difflib import SequenceMatcher
import urllib3

In [2]:
# Disable InsecureRequestWarning
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

Fetch Focus Products

In [3]:
df_allproducts = pd.read_csv('../../Data/products_data.csv')
df_allproducts

,brand_name,name,our_price
0,Mamaearth,Mamaearth Honey Malai Nourishing Bathing Soap ...,87.0
1,Plum,Plum BodyLovin' Caramel Popcorn Bathing Soap |...,70.0
2,Mamaearth,Mamaearth Nourishing Bathing Bar For Kids – 75gx5,439.0
3,Mamaearth,Mamaearth Tea Tree Nourishing Bathing Soap Wit...,337.0
4,Mamaearth,Mamaearth Natural Nourishing Bathing Soaps (5x...,322.0
...,...,...,...
4459,Blue Heaven,Blue Heaven Beauty blender,140.0
4460,Blue Heaven,Blue Heaven Super Blendable Make-Up Sponge,1.0
4461,Blue Heaven,Blue Heaven Super Blendable Make-Up Sponge,148.0
4462,Plum,Plum Bodylovin' Vanilla & Beachy Vibes Body Mi...,788.0


In [4]:
df_focusproducts = pd.read_excel('../../Data/Oct_targets Focus Products V1.xlsx')
df_focusproducts

,trim,ean code,Name,Brand Name,Category Name,Brand Hourse,Sum of nmv
0,30074576,30074576,Maybelline New York Volume Express Colossal Ma...,Mascara,Maybelline,L'Oreal,1.520144e+05
1,30079236,30079236,Maybelline New York The Colossal Volum Express...,Mascara,Maybelline,L'Oreal,1.667629e+06
2,30163188,30163188,L'Oreal Professionnel Serie ExpertA Metal DXA ...,Shampoo,L'Oreal Professionnel,L'Oreal Professionnel,3.157643e+04
3,30163362,30163362,L'Oreal Professionnel Metal DX Hair Mask | Ant...,Hair Mask,L'Oreal Professionnel,L'Oreal Professionnel,2.450879e+04
4,40062525,40062525,"NIVEA SOFT Light cream with Vitamin E, Jojoba ...",Face Moisturizers & Day Creams,Nivea,Nivea,1.014698e+05
...,...,...,...,...,...,...,...
5115,SQC00001,SQC00001,Stay Quirky Compact Powder For Fair Skin| Long...,Compact,Stay Quirky,PB,2.229797e+04
5116,SQLLS006,SQLLS006,Stay Quirky Liquid Lipstick |Transferproof| Lo...,Lipstick,Stay Quirky,PB,3.738438e+03
5117,SQLLS007,SQLLS007,"Stay Quirky Liquid Lipstick, Red - Too Hot For...",Lipstick,Stay Quirky,PB,1.885357e+04
5118,SQLLS015,SQLLS015,Stay Quirky Liquid Lipstick|Transferproof| Lon...,Lipstick,Stay Quirky,PB,2.615980e+03


In [5]:
focus_product_names = df_focusproducts['Name '].tolist()

# Filter df_allproducts to only include focus products
df_focusproducts_prices = df_allproducts[df_allproducts['name'].isin(focus_product_names)]

# Create a new DataFrame with focus product names and their prices
df_focusproducts_prices = df_focusproducts_prices[['name', 'our_price']].rename(columns={'name': 'Name', 'our_price': 'Price'})

# df_focusproducts_prices = df_focusproducts_prices.sort_values('Name')
df_focusproducts_prices = df_focusproducts_prices.reset_index(drop=True)

df_focusproducts_prices

,Name,Price
0,Mamaearth Honey Malai Nourishing Bathing Soap ...,87.0
1,Mamaearth Nourishing Bathing Bar For Kids – 75gx5,439.0
2,Mamaearth Vitamin C Nourishing Bathing Soap Wi...,315.0
3,mCaffeine Espresso Bathing Bar (100gm) for Dee...,239.0
4,mCaffeine Exfoliating Coffee Bathing Soap for ...,95.0
...,...,...
870,mCaffeine miniature Green Tea & Vitamin C Glow...,331.0
871,mCaffeine Coffee Moment Skin Care Gift Kit (30...,1199.0
872,mCaffeine Be Date Ready Body Polishing Gift Ki...,1002.0
873,Swiss Beauty Foundation Blender Brush,262.0


Run search queries on myntra and then performing scraping

In [6]:
def generate_myntra_url(product):
    clean = re.sub(r'[^a-zA-Z0-9. ]', '', product)
    newclean = []
    for i in clean.split():
        if i.isalpha():
            newclean.append(i)
    clean = ' '.join(newclean)

    query1 = clean.lower().replace(' ', '-')
    query2 = clean.replace(' ', '%20')

    url = f"https://www.myntra.com/{query1}?rawQuery={query2}"

    return url    

In [7]:
def scrape_myntra(product):
    # Define headers for requests
    headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}

    session = requests.Session()
    url = generate_myntra_url(product)
    response = session.get(url, headers=headers, allow_redirects=False)

    soup = BeautifulSoup(response.text, 'html.parser')

    script_element = soup.find_all('script', type='application/ld+json')
    required_data = None

    for script in script_element:
        json_data = json.loads(script.string)
        if json_data['@type'] == 'ItemList':
            required_data = json_data
            break

    possible_products = []

    try:
        for item in required_data['itemListElement']: # none type error if the page doesn't have products
            s = requests.Session()
            res = s.get(item['url'], headers=headers, verify=False)

            soup = BeautifulSoup(res.text,"html.parser")

            script = None
            for s in soup.find_all("script"):
                if 'pdpData' in s.text:
                    script = s.get_text(strip=True)
                    break

            data = json.loads(script[script.index('{'):])
            name = data['pdpData']['name']
            price = discounted_price = data["pdpData"]["sizes"][0]["sizeSellerData"][0]["discountedPrice"]
            possible_products.append({'name': name, 'price': price})
    except:
        pass

    return possible_products


Find a match from possible products

In [20]:
# Function to clean and preprocess product names
def clean_product_name(name):
    # Remove formatting and symbols
    name = re.sub(r'[^a-zA-Z0-9]', '', name)
    # Convert to lowercase and remove extra spaces
    name = name.lower()
    return name

# Function to calculate the similarity between two product names
def get_name_similarity(name1, name2):
    return SequenceMatcher(None, name1, name2).ratio()

def find_match(name, possible_products):
    max_similarity = 0
    match = None

    for product in possible_products:
        product_name = clean_product_name(product['name'])
        our_name = clean_product_name(name)


        similarity = get_name_similarity(product_name, our_name)

        if similarity > 0.85:
            if similarity > max_similarity:
                max_similarity = similarity
                match = product    
    return match

Find better priced products from our Focus Products

In [21]:
import csv
file = 'temp'

with open(f'{file}.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['purplle_name', 'myntra_name', 'purplle_price', 'myntra_price'])

    
    for index in range(10):
        row = df_focusproducts_prices.iloc[index]
        name, price = row["Name"], row["Price"]
        possible_products = scrape_myntra(name)

        match = find_match(name, possible_products)
        if match and match['price'] < price:
            writer.writerow([name, match['name'], price, match['price']])
